In [14]:
import torch 
import torchvision 
import numpy as np
import os 
import pandas as pd 
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch 
from torch.utils.data import Dataset
from PIL import Image

In [15]:
path="E:\\datasets\\yoga pose estimation\\"

In [16]:
test_df=pd.read_pickle(path+"datasettest_df_yoga.pkl")

In [17]:
train_df=pd.read_pickle(path+"datasettrain_yoga.pkl")

In [18]:
combined_df = pd.concat([train_df, test_df], ignore_index=True)

print(combined_df)

                             Label  \
0          eka pada rajakapotasana   
1                 bharadvajasana i   
2                    bhujapidasana   
3           urdhva mukha svanasana   
4                       anantasana   
...                            ...   
5622                    garudasana   
5623               viparita karani   
5624              urdhva hastasana   
5625  utthita ashwa sanchalanasana   
5626                      malasana   

                                                  Image  
0     <PIL.PngImagePlugin.PngImageFile image mode=RG...  
1     <PIL.PngImagePlugin.PngImageFile image mode=RG...  
2     <PIL.PngImagePlugin.PngImageFile image mode=RG...  
3     <PIL.PngImagePlugin.PngImageFile image mode=RG...  
4     <PIL.PngImagePlugin.PngImageFile image mode=RG...  
...                                                 ...  
5622  <PIL.PngImagePlugin.PngImageFile image mode=RG...  
5623  <PIL.PngImagePlugin.PngImageFile image mode=RG...  
5624  <PIL.PngImagePl

In [20]:
from torch.utils.data import Dataset ,DataLoader
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder

In [21]:
label_encoder=LabelEncoder()

In [22]:
combined_df["Label"]=label_encoder.fit_transform(combined_df["Label"])

In [23]:
num_classes=combined_df['Label'].nunique()

In [24]:
dataframe=combined_df

In [25]:
class CustomDataset(Dataset):
    def __init__ (self,dataframe,transform=None):
        self.data=dataframe
        self.transform=transform
    def __len__(self):

        return len(self.data)

        
    def __getitem__(self,index):

        label=self.data.iloc[index]["Label"]

        image=self.data.iloc[index]["Image"]

        if self.transform:
            image=self.transform(image)


        return image, label






In [26]:
from torchvision import transforms

In [27]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # RESNET İÇİN
    transforms.ToTensor(),  # Görüntüyü tensöre çeviriyoruz
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalizasyon
                         std=[0.229, 0.224, 0.225])
])

In [28]:
dataset=CustomDataset(dataframe=dataframe,transform=transform)

In [29]:
labels = dataset.data['Label'].values[:len(dataset)]  

indices = list(range(len(dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.2, stratify=labels)

train_dataset = torch.utils.data.Subset(dataset, train_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [30]:
import torchvision.models as models

In [32]:
model=models.squeezenet1_0(pretrained=True)

c:\Users\ninja\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ninja\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/squeezenet1_0-b66bff10.pth" to C:\Users\ninja/.cache\torch\hub\checkpoints\squeezenet1_0-b66bff10.pth
100%|██████████| 4.78M/4.78M [00:01<00:00, 4.62MB/s]


In [33]:
if torch.cuda.is_available():
    device=torch.device("cuda")

In [34]:
device
import torch.nn as nn

In [35]:
model

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [36]:
from torchsummary import summary 

In [39]:
model.classifier=nn.Sequential(
    nn.Dropout(p=0.5, inplace=False),
    nn.Conv2d(512,num_classes,kernel_size=(1,1),stride=(1,1)),
    nn.ReLU(inplace=True),
    nn.AdaptiveAvgPool2d(output_size=(1,1))
)



In [40]:
model


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [41]:
model.to(device)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [42]:
from torchsummary import summary 

In [43]:
summary(model,input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 96, 109, 109]          14,208
              ReLU-2         [-1, 96, 109, 109]               0
         MaxPool2d-3           [-1, 96, 54, 54]               0
            Conv2d-4           [-1, 16, 54, 54]           1,552
              ReLU-5           [-1, 16, 54, 54]               0
            Conv2d-6           [-1, 64, 54, 54]           1,088
              ReLU-7           [-1, 64, 54, 54]               0
            Conv2d-8           [-1, 64, 54, 54]           9,280
              ReLU-9           [-1, 64, 54, 54]               0
             Fire-10          [-1, 128, 54, 54]               0
           Conv2d-11           [-1, 16, 54, 54]           2,064
             ReLU-12           [-1, 16, 54, 54]               0
           Conv2d-13           [-1, 64, 54, 54]           1,088
             ReLU-14           [-1, 64,

In [44]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [48]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()  
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        # Forward pass
        outputs = model(images)
        labels=labels.long()
        #print(outputs.dtype)
        #print(labels.dtype)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/50], Loss: 3.1797
Epoch [2/50], Loss: 3.0403
Epoch [3/50], Loss: 2.8641
Epoch [4/50], Loss: 2.7214
Epoch [5/50], Loss: 2.6395
Epoch [6/50], Loss: 2.4833
Epoch [7/50], Loss: 2.3649
Epoch [8/50], Loss: 2.2587
Epoch [9/50], Loss: 2.1612
Epoch [10/50], Loss: 2.0436
Epoch [11/50], Loss: 1.9660
Epoch [12/50], Loss: 1.8580
Epoch [13/50], Loss: 1.7797
Epoch [14/50], Loss: 1.6793
Epoch [15/50], Loss: 1.6082
Epoch [16/50], Loss: 1.5118
Epoch [17/50], Loss: 1.4217
Epoch [18/50], Loss: 1.4083
Epoch [19/50], Loss: 1.3311
Epoch [20/50], Loss: 1.2554
Epoch [21/50], Loss: 1.1905
Epoch [22/50], Loss: 1.0974
Epoch [23/50], Loss: 1.0728
Epoch [24/50], Loss: 1.0356
Epoch [25/50], Loss: 0.9713
Epoch [26/50], Loss: 0.9557
Epoch [27/50], Loss: 0.8700
Epoch [28/50], Loss: 0.8412
Epoch [29/50], Loss: 0.7933
Epoch [30/50], Loss: 0.8002
Epoch [31/50], Loss: 0.7675
Epoch [32/50], Loss: 0.6960
Epoch [33/50], Loss: 0.6844
Epoch [34/50], Loss: 0.6400
Epoch [35/50], Loss: 0.6435
Epoch [36/50], Loss: 0.5852
E

In [49]:
torch.save(model.state_dict(), 'squuze_net_finetuned_yoga.pth')


In [50]:
torch.save(model.state_dict(), 'model_checkpoint_squuze_net.pth')


In [51]:
model.eval()   
correct = 0
total = 0

with torch.no_grad():   
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 44.94%
